**Wrangling Data from Laboratory Reports**

Python is a good programming language choice for the medical director/data scientist who needs to parse text from lab reports, paramedical exams, and underwriting notes. While some data sources require sophisticated programming techniques, the parsing of many common sources such as laboratory results is well within the reach of a medical director who is a programming novice.

In [ ]:
print("Hello")

  fgads,f; jkasdlfjasldkj
  jsklJSLKFJ

In [ ]:
TARGETDIR = r'./AbLab_Rpts'
import os

start_directory = TARGETDIR  # Start from the target directory

print(f"Walking through directory: {os.path.abspath(start_directory)}\n")
# Iterate over the 3-tuples generated by os.walk()
for dirpath, dirnames, filenames in os.walk(start_directory):
    # Print the current directory path
    print(f"Current Directory: {dirpath}")
    
    # List subdirectories found
    if dirnames:
        print(f"  Subdirectories: {', '.join(dirnames)}")

    # Iterate over files in the current directory
    for filename in filenames:
        # Construct and print the full path of each file
        full_file_path = os.path.join(dirpath, filename)
        print(f"  Found file: {full_file_path}")
    print("-" * 40)
    